In [1]:
import os
os.chdir(os.path.expanduser("~/work/nbs_pipeline"))

In [2]:
#| export
verbose                       = 1  
show_plots                    = False
reset_kernel                  = False
pre_configured_case           = True 
case_id                       = 7 
frequency_factor              = 1
frequency_factor_change_alias = True 
cuda_device                   = 0

In [3]:
#| export
import pandas as pd
import numpy as np
from fastcore.all import *
import wandb
from dvats.load import TSArtifact, infer_or_inject_freq
import pickle
import matplotlib
import matplotlib.pyplot as plt
from tsai.data.external import convert_tsf_to_dataframe
from tsai.utils import stack_pad
#| export
import warnings
warnings.filterwarnings("ignore", module="umap")
import os
import sys
sys.path.append(os.path.abspath('..'))
from dvats.all import *
from fastcore.all import *
from tsai.basics import *
from tsai.models.InceptionTimePlus import *
from tsai.callback.MVP import *
import matplotlib.colors as colors
from fastai.callback.wandb import WandbCallback
from fastai.callback.progress import ShowGraphCallback
from fastai.callback.schedule import *
from fastai.callback.tracker import EarlyStoppingCallback
import wandb

import torch
import matplotlib.pyplot as plt
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from huggingface_hub import hf_hub_download
import sys
import dvats.utils as ut
import torch
import matplotlib.pyplot as plt
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from huggingface_hub import hf_hub_download

import pyarrow.feather as ft
from gluonts.transform.split import TFTInstanceSplitter
from gluonts.transform.sampler import TestSplitSampler
import numpy as np
import einops
import torch.nn.functional as F
from dvats.memory import gpu_memory_status


Octave is ready <oct2py.core.Oct2Py object at 0x7f85b520a830>

















In [4]:
import dvats.config as cfg_

In [5]:
#| export
user, project, version, data, config, job_type = cfg_.get_artifact_config_MVP(False)
if pre_configured_case: 
    cfg_.force_artifact_config_mvp(
        config = config,
        id = case_id,
        verbose = verbose, 
        both = verbose > 0,
        frequency_factor = frequency_factor,
        frequency_factor_change_alias = frequency_factor_change_alias
    )
#| export
path = os.path.expanduser("~/work/nbs_pipeline/")
name="02c_encoder_moment-embedding"
os.environ["WANDB_NOTEBOOK_NAME"] = path+name+".ipynb"
runname=name
if verbose > 0: print("runname: "+runname)
if verbose > 0: cfg_.show_attrdict(config)

#| export
if verbose > 0: print("--> Wandb init")
run = wandb.init(
    entity = user,
    # work-nbs is a place to log draft runs
    project=project,
    group=config.wandb_group,
    job_type=job_type,
    allow_val_change=True,
    mode=config.analysis_mode,
    config=config,
    # When use_wandb is false the run is not linked to a personal account
    #NOTE: This is not working right now
    anonymous = 'never' if config.use_wandb else 'must', 
    resume=False,
    name = runname
)
if verbose > 0: print("Wandb init -->")
config = run.config  # Object for storing hyperparameters
artifacts_gettr = run.use_artifact if config.use_wandb else wandb_api.artifact
#| export
config = run.config  # Object for storing hyperparameters
if verbose > 0: cfg_.show_attrdict(config)
artifacts_gettr = run.use_artifact if config.use_wandb else wandb_api.artifact
train_artifact = artifacts_gettr(config.train_artifact)
if verbose > 0: print("---> W&B Train Artifact")

#| export
import pyarrow.feather as ft
df_train = train_artifact.to_df()
print("df_train ~", df_train.shape)
print(df_train.shape)
display(df_train.head())
#| export
if verbose > 0: print("---> Sliding window | ", config.w,  " | ", config.stride )
sw = SlidingWindow(window_len=config.w, stride=config.stride, get_y=[])
if verbose > 0: print(" Sliding window | ", config.w,  " | ", config.stride, "---> | df_train ~ ", df_train.shape )
X_train, _ = sw(df_train)
if verbose > 0: print(" sw_df_train | ", config.w,  " | ", config.stride, "--->" )
print("X_train ~", X_train.shape)

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find /home/macu/work/nbs_pipeline/02c_encoder_moment-embedding.ipynb.


train_artifact: mi-santamaria/deepvats/PulsusParadoxus-SP02:latest -> mi-santamaria/deepvats/toy:latest
stride: 900 -> 1
epochs: 100
mask_sync: False
time_col is missing in original dict | None 
analysis_mode: online
mask_stateful: True
valid_artifact: None
mvp_ws: (15, 100) -> [10, 30]
valid_size: 0.2
data_cols is missing in original dict | [] 
mask_future: False
batch_size: 512 -> 32
norm_by_sample: False
use_wandb: True
csv_config is missing in original dict | {} 
freq is missing in original dict | 1s 
norm_use_single_batch: False
artifact_name is missing in original dict | toy 
w: 100 -> 30
r: 0.71
data_fpath is missing in original dict | ~/data/toy.csv 
alias: PulsusParadoxus-SP02 -> toy
norm_use_by_single_batch is missing in original dict | (False,) 
wandb_group: None
runname: 02c_encoder_moment-embedding
alias: toy
analysis_mode: online
batch_size: 32
epochs: 100
mask_future: False
mask_stateful: True
mask_sync: False
mvp_ws: [10, 30]
norm_by_sample: False
norm_use_single_batch:

wandb: Currently logged in as: mi-santamaria. Use `wandb login --relogin` to force relogin


Wandb init -->
alias: toy
analysis_mode: online
batch_size: 32
epochs: 100
mask_future: False
mask_stateful: True
mask_sync: False
mvp_ws: [10, 30]
norm_by_sample: False
norm_use_single_batch: False
r: 0.71
stride: 1
train_artifact: mi-santamaria/deepvats/toy:latest
valid_artifact: None
use_wandb: True
valid_size: 0.2
w: 30
wandb_group: None
artifact_name: toy
data_cols: []
data_fpath: ~/data/toy.csv
freq: 1s
time_col: None
csv_config: {}
norm_use_by_single_batch: [False]
---> W&B Train Artifact


wandb:   1 of 1 files downloaded.  


df_train ~ (550, 3)
(550, 3)


,T3,T2,T1
1970-01-01 00:00:00,0.741822,0.637180,0.565117
1970-01-01 00:00:01,0.739731,0.629415,0.493513
1970-01-01 00:00:02,0.718757,0.539220,0.469350
1970-01-01 00:00:03,0.730169,0.577670,0.444100
1970-01-01 00:00:04,0.752406,0.570180,0.373008


---> Sliding window |  30  |  1
 Sliding window |  30  |  1 ---> | df_train ~  (550, 3)
 sw_df_train |  30  |  1 --->
X_train ~ (521, 3, 30)


In [6]:
X_train.shape

(521, 3, 30)

## Siguiendo el tutorial

### Descarga del modelo

In [35]:
lib_path = os.getenv("LIB_PATH")

In [19]:
repo_id = "Melady/Tempo"
repo_dir = repo_id + " etth2_336_96_checkpoint.pth"
local_path = lib_path + "/TEMPO"
local_dir = local_path + "/TEMPO_checkpoints"

In [36]:
print("repo: ", repo_dir)
print("local: ", local_dir)

repo:  Melady/Tempo etth2_336_96_checkpoint.pth
local:  /usr/local/share/lib/TEMPO/TEMPO_checkpoints


In [38]:
#!huggingface-cli download Melady/TEMPO etth2_336_96_checkpoint.pth --local-dir {local_dir}

In [44]:
import subprocess

#try:
    # Intentar descargar con huggingface-cli
#    subprocess.run(
#        f"huggingface-cli download {repo_dir} --local-dir {local_dir}",
#        check=True,
#        shell=True
#    )
#except subprocess.CalledProcessError:
    # Si el primer comando falla, intentar descargar con wget
    #wget_url = "https://cdn-lfs-us-1.hf.co/repos/2c/dc/2cdcba67a0608415e4b776ac55aebcca6cac1f6807cf2cd4985e3f0d18ca51f8/7525f9f1c9cfa5048556c345c595f112437c98d5307d5a85ea3b0c482ecf5032?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27etth2_336_96_checkpoint.pth%3B+filename%3D%22etth2_336_96_checkpoint.pth%22%3B&Expires=1729171448&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyOTE3MTQ0OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzJjL2RjLzJjZGNiYTY3YTA2MDg0MTVlNGI3NzZhYzU1YWViY2NhNmNhYzFmNjgwN2NmMmNkNDk4NWUzZjBkMThjYTUxZjgvNzUyNWY5ZjFjOWNmYTUwNDg1NTZjMzQ1YzU5NWYxMTI0MzdjOThkNTMwN2Q1YTg1ZWEzYjBjNDgyZWNmNTAzMj9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=OHcFfIIQ5pHCoBiCCOd348TyjH1dweCGZpv0uHYu2gwwu5bolKZzEu9gIM8FquzwNa37Mz7DoiwyXdEo9yGHviX~jcxKvX5TRq4vq4NuUBXfnoMBgh~sWVwdsu7sTjxqOHUkKCoquJPJ5xa0qRJ8nb9s7szgzk5XH55W6QA954bejd1kW6gEaG2Ff3qFP7JTQ5SgGMwodh~-mJDlKCh02UpcOZxIsMdb~SIhk98tbR0I492p74plX~ZquN55TJDSwH~Yxa-izULLHA2zM-sxbMTa9HGAL2yg93RBK-~c~TJgXwUn1jEgpVTloyzO5xR4q6UEJPzapcWqetqsu1TpNA__&Key-Pair-Id=K24J24Z295AEI9"  # Coloca aquí la URL completa
    #subprocess.run(
    #    f"wget \"{wget_url}\" -O {local_dir}/etth2_336_96_checkpoint.pth",
    #    check=True,
    #    shell=True
    #)

#### Required package & baseline models

In [45]:
from tempo.data_provider.data_factory import data_provider
from tempo.utils.tools import EarlyStopping, adjust_learning_rate, visual, vali, test
from torch.utils.data import Subset
from tqdm import tqdm
from tempo.models.PatchTST import PatchTST
from tempo.models.GPT4TS import GPT4TS
from models.DLinear import DLinear
from models.TEMPO import TEMPO
# from models.T5 import T54TS
from models.ETSformer import ETSformer


import numpy as np
import torch
import torch.nn as nn
from torch import optim
from numpy.random import choice

import os
import time

import warnings
import matplotlib.pyplot as plt
import numpy as np

import argparse
import random
import sys

from omegaconf import OmegaConf

import torch.nn as nn
from torch import optim
from numpy.random import choice

import os
import time

import warnings
import matplotlib.pyplot as plt
import numpy as np

import argparse
import random
import sys

from omegaconf import OmegaConf

ModuleNotFoundError: No module named 'data_provider'